В прошлых сериях чтобы обучить агента мы собирали максимум информации о среде, проходя по всем доступным состояниям с помощью матрицы переходов $T$. Сбор этой информации обходится дорого даже если ограничивать себя стратегией $\pi$. Хуже того, матрица переходов не всегда известна заранее! Чтобы преодолеть эти трудности нужен новый подход. 

Замечание. Алгоритмы, требующие для работы информацию об окружающей среде, называются **основанными на модели**, а алгоритмы не нуждающиеся в априорной информации о переходах и вознаграждениях называются **безмодельными**

Рассмотрим безмодельный метод Монте-Карло, основанный на рандомизации данных: мы строим выборку, анализируем её свойства и делам выводы о распределении, из которого она была извлечена. Чем больше итераций, тем точнее приближается искомое распределение. 

Итак, у нас есть игра, но нет данных о матрице переходов и вознаграждений. Полное вознаграждение (или доход), полученное в течение эпизода, начиная с момента времени t, выражается формулой

$G_t = \sum_k \gamma^k R_{t+k+1}$

Но теперь его оценка имеет эмпирический характер, т.к. строится на основе множества выборок. 

Замечание: $G_t$ удобнее вычислять от конца к началу. И это в общем-то достаточно интуитивно, если помнить, что занимаемся планированием и считаем шаги наперёд. 

In [ ]:
import torch
import gym

env = gym.make('FrozenLake-v0')

gamma = 1
n_episode = 10000

# оптимальная стратегия для игры FrozenLake, вычисленная в предыдущих сериях
optimal_policy = torch.tensor([0., 3., 3., 3., 0., 3., 2., 3., 3., 1., 0., 3., 3., 2., 1., 3.])

In [ ]:
def run_episode(env, policy):
    state = env.reset()
    rewards = []
    states = [state]
    is_done = False
    while not is_done:
        action = policy[state].item()
        state, reward, is_done, info = env.step(action)
        states.append(state)
        rewards.append(reward)
        if is_done:
            break
    states = torch.tensor(states)
    rewards = torch.tensor(rewards)
    return states, rewards

Дополнительно позникает интересный вопрос: если состояние s в течение эпизода посещается несколько раз, сколько раз стоит учитывать его при вычислении функции ценности? Основных подходов два: учесть самое первое посещение и усреднить по всем посещениям в рамках эпизода. Рассмотрим, как считается каждый из вариантов:

In [ ]:
def mc_prediction_first_visit(env, policy, gamma, n_episode):
    n_state = policy.shape[0]
    V = torch.zeros(n_state)
    N = torch.zeros(n_state)
    for episode in range(n_episode):
        states_t, rewards_t = run_episode(env, policy)
        return_t = 0
        first_visit = torch.zeros(n_state)
        G = torch.zeros(n_state)
        for state_t, reward_t in zip(reversed(states_t)[1:], reversed(rewards_t)):
            return_t = gamma * return_t + reward_t
            G[state_t] = return_t
            first_visit[state_t] = 1
        for state in range(n_state):
            if first_visit[state] > 0:
                V[state] += G[state]
                N[state] += 1
    for state in range(n_state):
        if N[state] > 0:
            V[state] = V[state] / N[state]
    return V

In [ ]:
value = mc_prediction_first_visit(env, optimal_policy, gamma, n_episode)

print('The value function calculated by first-visit MC prediction:\n', value)

In [ ]:
def mc_prediction_every_visit(env, policy, gamma, n_episode):
    n_state = policy.shape[0]
    V = torch.zeros(n_state)
    N = torch.zeros(n_state)
    G = torch.zeros(n_state)
    for episode in range(n_episode):
        states_t, rewards_t = run_episode(env, policy)
        return_t = 0
        for state_t, reward_t in zip(reversed(states_t)[1:], reversed(rewards_t)):
            return_t = gamma * return_t + reward_t
            G[state_t] += return_t
            N[state_t] += 1
    for state in range(n_state):
        if N[state] > 0:
            V[state] = G[state] / N[state]
    return V

In [ ]:
value = mc_prediction_every_visit(env, optimal_policy, gamma, n_episode)

print('The value function calculated by every-visit MC prediction:\n', value)